In [ ]:
from typing import Optional, List
import time

class PromptOptimizer:
    """
    Main orchestrator for prompt evaluation and iterative improvement.
    Tracks best prompt across all iterations.
    """
    
    def __init__(self, metric_config: dict, config: dict, api_key: str = None):
        self.metric_config = metric_config
        self.config = config
        
        # Initialize components
        self.evaluator = PromptEvaluator(metric_config, config, api_key)
        self.improver = PromptImprover(metric_config, config, api_key)
        
        # Iteration tracking
        self.history = []
        self.best_result = None
    
    def optimize(self, initial_prompt: str, max_iterations: int = None,
                 target_score: float = None, min_improvement: float = None) -> Dict:
        """
        Optimize a prompt through iterative evaluation and improvement.
        
        Args:
            initial_prompt: The starting prompt to optimize
            max_iterations: Maximum iterations (default from config)
            target_score: Target score to reach (default from config)
            min_improvement: Minimum improvement to continue (default from config)
        
        Returns:
            Dictionary with optimization results and best prompt
        """
        # Use config defaults if not specified
        max_iterations = max_iterations or self.config['max_iterations']
        target_score = target_score or self.config['target_score']
        min_improvement = min_improvement or self.config['min_improvement']
        
        print("\n" + "="*80)
        print("PROMPT OPTIMIZATION STARTED")
        print("="*80)
        print(f"Max Iterations: {max_iterations}")
        print(f"Target Score: {target_score}/{self.config['max_score']} ({target_score/self.config['max_score']*100:.0f}%)")
        print(f"Min Improvement: {min_improvement} points")
        print()
        
        # Reset tracking
        self.history = []
        self.best_result = None
        
        current_prompt = initial_prompt
        iteration = 0
        
        # Main optimization loop
        while iteration < max_iterations:
            iteration += 1
            version = f"v{iteration}"
            
            print(f"\n{'='*80}")
            print(f"ITERATION {iteration}/{max_iterations} ({version})")
            print(f"{'='*80}")
            
            # Evaluate current prompt
            is_first_run = (iteration == 1)
            
            if is_first_run:
                # First iteration: evaluate all metrics
                eval_result = self.evaluator.evaluate(current_prompt, first_run=True)
            else:
                # Subsequent iterations: only re-evaluate below-threshold metrics
                previous_below_threshold = self.history[-1]['evaluation']['below_threshold_metrics']
                eval_result = self.evaluator.evaluate(
                    current_prompt, 
                    first_run=False,
                    below_threshold_metrics=previous_below_threshold
                )
            
            # Store iteration data
            iteration_data = {
                'version': version,
                'iteration': iteration,
                'prompt': current_prompt,
                'evaluation': eval_result,
                'score': eval_result['total_score'],
                'percentage': eval_result['percentage'],
                'below_threshold_count': eval_result['below_threshold_count']
            }
            self.history.append(iteration_data)
            
            # Update best result if this is better
            if self.best_result is None or eval_result['total_score'] > self.best_result['score']:
                self.best_result = {
                    'version': version,
                    'iteration': iteration,
                    'prompt': current_prompt,
                    'evaluation': eval_result,
                    'score': eval_result['total_score'],
                    'percentage': eval_result['percentage']
                }
                print(f"\n🏆 NEW BEST SCORE: {eval_result['total_score']:.2f}/150 ({eval_result['percentage']:.1f}%)")
            
            # Check stopping conditions
            if eval_result['total_score'] >= target_score:
                print(f"\n✅ TARGET SCORE REACHED: {eval_result['total_score']:.2f} >= {target_score}")
                print(f"Stopping after {iteration} iterations.")
                break
            
            if eval_result['below_threshold_count'] == 0:
                print(f"\n✅ ALL METRICS ABOVE THRESHOLD!")
                print(f"Stopping after {iteration} iterations.")
                break
            
            # Check if we should continue
            if iteration >= max_iterations:
                print(f"\n⏹️  MAX ITERATIONS REACHED ({max_iterations})")
                break
            
            # Check for minimal improvement
            if iteration > 1:
                previous_score = self.history[-2]['score']
                score_change = eval_result['total_score'] - previous_score
                
                if score_change < min_improvement:
                    print(f"\n⏹️  MINIMAL IMPROVEMENT: +{score_change:.2f} points < {min_improvement} threshold")
                    print(f"Stopping after {iteration} iterations.")
                    break
            
            # Generate improved prompt for next iteration
            print(f"\n{'─'*80}")
            print(f"Generating improved prompt for next iteration...")
            print(f"{'─'*80}")
            
            improvement_result = self.improver.improve_prompt(current_prompt, eval_result)
            
            # Store improvement data
            iteration_data['improvement'] = improvement_result
            
            print(f"\n📝 Changes planned:")
            changes_preview = improvement_result['change_explanations'][:300]
            print(changes_preview + "..." if len(improvement_result['change_explanations']) > 300 else changes_preview)
            
            # Update current prompt for next iteration
            current_prompt = improvement_result['improved_prompt']
        
        # Optimization complete
        return self._compile_optimization_results(iteration)
    
    def _compile_optimization_results(self, total_iterations: int) -> Dict:
        """Compile final optimization results."""
        
        print("\n" + "="*80)
        print("OPTIMIZATION COMPLETE")
        print("="*80)
        
        # Calculate improvement statistics
        first_score = self.history[0]['score']
        final_score = self.history[-1]['score']
        best_score = self.best_result['score']
        
        total_improvement = best_score - first_score
        improvement_pct = (total_improvement / first_score * 100) if first_score > 0 else 0
        
        print(f"\n📊 Summary:")
        print(f"   Total Iterations: {total_iterations}")
        print(f"   First Score: {first_score:.2f}/150 ({self.history[0]['percentage']:.1f}%)")
        print(f"   Final Score: {final_score:.2f}/150 ({self.history[-1]['percentage']:.1f}%)")
        print(f"   Best Score: {best_score:.2f}/150 ({self.best_result['percentage']:.1f}%) - {self.best_result['version']}")
        print(f"   Total Improvement: +{total_improvement:.2f} points (+{improvement_pct:.1f}%)")
        
        if self.best_result['version'] != f"v{total_iterations}":
            print(f"\n⚠️  Note: Best prompt is from {self.best_result['version']}, not final iteration")
        
        # Compile results
        results = {
            'total_iterations': total_iterations,
            'history': self.history,
            'best_result': self.best_result,
            'first_score': first_score,
            'final_score': final_score,
            'best_score': best_score,
            'total_improvement': total_improvement,
            'improvement_percentage': improvement_pct,
            'target_reached': best_score >= self.config['target_score']
        }
        
        return results
    
    def get_best_prompt(self) -> Optional[str]:
        """Get the best prompt from all iterations."""
        if self.best_result:
            return self.best_result['prompt']
        return None
    
    def get_improvement_trajectory(self) -> List[Dict]:
        """Get score trajectory across iterations."""
        return [
            {
                'iteration': h['iteration'],
                'version': h['version'],
                'score': h['score'],
                'percentage': h['percentage'],
                'below_threshold': h['below_threshold_count']
            }
            for h in self.history
        ]
    
    def generate_optimization_report(self, results: Dict) -> str:
        """Generate comprehensive optimization report."""
        
        lines = []
        lines.append("="*100)
        lines.append("PROMPT OPTIMIZATION REPORT")
        lines.append("="*100)
        lines.append("")
        
        # Summary
        lines.append("SUMMARY")
        lines.append("-"*100)
        lines.append(f"Total Iterations: {results['total_iterations']}")
        lines.append(f"Starting Score: {results['first_score']:.2f}/150 ({self.history[0]['percentage']:.1f}%)")
        lines.append(f"Final Score: {results['final_score']:.2f}/150 ({self.history[-1]['percentage']:.1f}%)")
        lines.append(f"Best Score: {results['best_score']:.2f}/150 ({results['best_result']['percentage']:.1f}%) "
                    f"[{results['best_result']['version']}]")
        lines.append(f"Total Improvement: +{results['total_improvement']:.2f} points "
                    f"(+{results['improvement_percentage']:.1f}%)")
        lines.append(f"Target Reached: {'✅ Yes' if results['target_reached'] else '❌ No'}")
        lines.append("")
        
        # Iteration-by-iteration breakdown
        lines.append("ITERATION HISTORY")
        lines.append("-"*100)
        lines.append(f"{'Version':<10} {'Score':<15} {'Change':<15} {'Below Threshold':<20} {'Status':<20}")
        lines.append("-"*100)
        
        for i, iteration in enumerate(self.history):
            version = iteration['version']
            score = iteration['score']
            below = iteration['below_threshold_count']
            
            if i == 0:
                change_str = "Initial"
            else:
                change = score - self.history[i-1]['score']
                change_str = f"+{change:.2f}" if change >= 0 else f"{change:.2f}"
            
            is_best = (version == results['best_result']['version'])
            status = "🏆 BEST" if is_best else ""
            
            lines.append(f"{version:<10} {score:.2f}/150{'':<5} {change_str:<15} {below} metrics{'':<10} {status:<20}")
        
        lines.append("")
        
        # Best prompt details
        lines.append("="*100)
        lines.append(f"BEST PROMPT ({results['best_result']['version']})")
        lines.append("="*100)
        lines.append("")
        lines.append(results['best_result']['prompt'])
        lines.append("")
        
        # Best prompt metrics breakdown
        lines.append("="*100)
        lines.append("BEST PROMPT EVALUATION BREAKDOWN")
        lines.append("="*100)
        
        best_eval = results['best_result']['evaluation']
        for group_name, group_data in best_eval['group_scores'].items():
            status = "✅" if group_data['percentage'] >= 70 else "⚠️"
            lines.append(f"\n{status} {group_name.upper().replace('_', ' ')}: "
                        f"{group_data['score']:.2f}/{group_data['max']} ({group_data['percentage']:.1f}%)")
        
        lines.append("")
        
        # Improvements made across all iterations
        lines.append("="*100)
        lines.append("IMPROVEMENTS MADE ACROSS ITERATIONS")
        lines.append("="*100)
        
        for i, iteration in enumerate(self.history):
            if 'improvement' in iteration:
                lines.append(f"\n{iteration['version']} → v{i+2}:")
                lines.append("-"*100)
                lines.append(iteration['improvement']['change_explanations'])
        
        return "\n".join(lines)


print("✅ Prompt Optimizer (Iteration Manager) Created!")

In [ ]:
def test_optimization_pipeline(api_key: str = None, use_mock: bool = True):
    """Test the complete optimization pipeline."""
    
    # Test with different complexity levels
    test_cases = {
        "simple": "Write a blog post about AI",
        
        "moderate": """
Write a technical article about machine learning.
Include examples and keep it under 2000 words.
""",
        
        "structured": """
# Task: Technical Guide

Create a guide comparing transformer architectures.

Requirements:
1. Explain BERT, GPT, and T5
2. Include code examples
3. Target audience: ML engineers

Keep it technically accurate.
"""
    }
    
    # Choose test case
    test_name = "moderate"
    test_prompt = test_cases[test_name]
    
    print(f"Testing Optimization Pipeline: '{test_name}' prompt")
    print(f"Using {'MOCK' if use_mock else 'REAL'} evaluation/improvement")
    print()
    
    # Initialize optimizer
    if use_mock:
        optimizer = PromptOptimizer(METRIC_CONFIG, CONFIG, api_key=None)
        
        # Replace with mock versions
        def mock_eval(prompt, first_run=True, below_threshold_metrics=None):
            # Call custom evaluator (real)
            custom_results = optimizer.evaluator.custom_evaluator.evaluate_all(prompt)
            
            # Mock LLM evaluation
            llm_metrics = {
                name: config for name, config in METRIC_CONFIG.items()
                if config['type'] == 'llm_judge'
            }
            if not first_run and below_threshold_metrics:
                llm_metrics = {
                    name: config for name, config in llm_metrics.items()
                    if name in below_threshold_metrics
                }
            
            llm_results = mock_llm_evaluation(prompt, list(llm_metrics.keys()))
            
            all_results = {**custom_results, **llm_results}
            return optimizer.evaluator._compile_results(prompt, all_results, first_run)
        
        optimizer.evaluator.evaluate = mock_eval
        
        def mock_improve(original, eval_result):
            return _mock_improve_prompt(original, eval_result)
        
        optimizer.improver.improve_prompt = mock_improve
    else:
        if not api_key:
            print("⚠️  API key required for real optimization")
            return None
        optimizer = PromptOptimizer(METRIC_CONFIG, CONFIG, api_key=api_key)
    
    # Run optimization
    start_time = time.time()
    results = optimizer.optimize(test_prompt)
    elapsed_time = time.time() - start_time
    
    print(f"\n⏱️  Total Time: {elapsed_time:.2f} seconds")
    
    # Generate report
    print("\n" + "="*100)
    print("GENERATING OPTIMIZATION REPORT")
    print("="*100)
    report = optimizer.generate_optimization_report(results)
    
    # Show report preview
    report_lines = report.split('\n')
    print('\n'.join(report_lines[:80]))  # Show first 80 lines
    if len(report_lines) > 80:
        print(f"\n... ({len(report_lines) - 80} more lines in full report) ...")
    
    return optimizer, results, report


# Run test
print("="*100)
print("RUNNING FULL OPTIMIZATION PIPELINE TEST")
print("="*100)

optimizer, opt_results, opt_report = test_optimization_pipeline(use_mock=True)

if opt_results:
    print("\n✅ Optimization complete!")
    print(f"   Optimizer stored in 'optimizer' variable")
    print(f"   Results stored in 'opt_results' variable")
    print(f"   Report stored in 'opt_report' variable")

In [ ]:
def visualize_optimization_progress(results: Dict):
    """Visualize the optimization progress across iterations."""
    
    history = results['history']
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Prompt Optimization Progress', fontsize=16, fontweight='bold')
    
    # 1. Score Progression
    ax1 = axes[0, 0]
    iterations = [h['iteration'] for h in history]
    scores = [h['score'] for h in history]
    
    ax1.plot(iterations, scores, marker='o', linewidth=2, markersize=8, color='steelblue')
    ax1.axhline(y=CONFIG['target_score'], color='green', linestyle='--', 
                label=f'Target ({CONFIG["target_score"]})', linewidth=2)
    
    # Highlight best
    best_iter = results['best_result']['iteration']
    best_score = results['best_result']['score']
    ax1.scatter([best_iter], [best_score], color='gold', s=300, marker='*', 
                zorder=5, label='Best', edgecolors='black', linewidth=2)
    
    ax1.set_xlabel('Iteration', fontsize=12)
    ax1.set_ylabel('Total Score (out of 150)', fontsize=12)
    ax1.set_title('Score Progression', fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    ax1.set_xticks(iterations)
    
    # 2. Below-Threshold Metrics Count
    ax2 = axes[0, 1]
    below_counts = [h['below_threshold_count'] for h in history]
    
    colors = ['red' if c > 0 else 'green' for c in below_counts]
    ax2.bar(iterations, below_counts, color=colors, alpha=0.7, edgecolor='black')
    ax2.set_xlabel('Iteration', fontsize=12)
    ax2.set_ylabel('Metrics Below Threshold', fontsize=12)
    ax2.set_title('Metrics Needing Improvement', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3, axis='y')
    ax2.set_xticks(iterations)
    
    # 3. Score Change Per Iteration
    ax3 = axes[1, 0]
    score_changes = [0]  # First iteration has no change
    for i in range(1, len(history)):
        change = history[i]['score'] - history[i-1]['score']
        score_changes.append(change)
    
    colors_change = ['green' if c >= 0 else 'red' for c in score_changes]
    bars = ax3.bar(iterations, score_changes, color=colors_change, alpha=0.7, edgecolor='black')
    ax3.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax3.set_xlabel('Iteration', fontsize=12)
    ax3.set_ylabel('Score Change', fontsize=12)
    ax3.set_title('Score Improvement Per Iteration', fontsize=14, fontweight='bold')
    ax3.grid(True, alpha=0.3, axis='y')
    ax3.set_xticks(iterations)
    
    # Add value labels on bars
    for bar, change in zip(bars, score_changes):
        if change != 0:
            height = bar.get_height()
            label_y = height + (0.5 if height > 0 else -1.5)
            ax3.text(bar.get_x() + bar.get_width()/2., label_y,
                    f'{change:+.1f}', ha='center', va='bottom' if height > 0 else 'top',
                    fontsize=9, fontweight='bold')
    
    # 4. Group Scores Heatmap (for best version)
    ax4 = axes[1, 1]
    best_eval = results['best_result']['evaluation']
    groups = list(best_eval['group_scores'].keys())
    group_percentages = [best_eval['group_scores'][g]['percentage'] for g in groups]
    
    # Create color map based on percentage
    colors_heat = []
    for pct in group_percentages:
        if pct >= 80:
            colors_heat.append('darkgreen')
        elif pct >= 70:
            colors_heat.append('green')
        elif pct >= 50:
            colors_heat.append('orange')
        else:
            colors_heat.append('red')
    
    y_pos = range(len(groups))
    bars = ax4.barh(y_pos, group_percentages, color=colors_heat, alpha=0.7, edgecolor='black')
    ax4.axvline(x=70, color='red', linestyle='--', label='Threshold (70%)', linewidth=2)
    ax4.set_yticks(y_pos)
    ax4.set_yticklabels([g.replace('_', ' ').title() for g in groups], fontsize=10)
    ax4.set_xlabel('Percentage', fontsize=12)
    ax4.set_title(f'Best Version ({results["best_result"]["version"]}) - Group Scores', 
                  fontsize=14, fontweight='bold')
    ax4.legend()
    ax4.grid(True, alpha=0.3, axis='x')
    ax4.set_xlim(0, 100)
    
    # Add percentage labels
    for bar, pct in zip(bars, group_percentages):
        width = bar.get_width()
        ax4.text(width + 2, bar.get_y() + bar.get_height()/2.,
                f'{pct:.1f}%', ha='left', va='center', fontsize=9, fontweight='bold')
    
    plt.tight_layout()
    return fig


# Visualize optimization results
if 'opt_results' in globals():
    print("\n" + "="*100)
    print("GENERATING OPTIMIZATION VISUALIZATION")
    print("="*100)
    fig = visualize_optimization_progress(opt_results)
    plt.show()
    print("✅ Visualization complete!")

# Step 7: Results, Visualization & Demo Package

In [ ]:
import os
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import pandas as pd

class PromptOptimizationDemo:
    """
    Complete demo package for prompt optimization system.
    Provides easy-to-use interface with visualization and export capabilities.
    """
    
    def __init__(self, api_key: str = None, use_mock: bool = True):
        """
        Initialize the demo system.
        
        Args:
            api_key: Anthropic API key (optional, required for real evaluation)
            use_mock: If True, use mock evaluation for demo purposes
        """
        self.use_mock = use_mock
        self.api_key = api_key
        
        # Initialize optimizer
        if use_mock:
            print("🔧 Initializing in MOCK mode (no API calls)")
            self.optimizer = PromptOptimizer(METRIC_CONFIG, CONFIG, api_key=None)
            self._setup_mock_mode()
        else:
            if not api_key:
                raise ValueError("API key required when use_mock=False")
            print("🔧 Initializing in REAL mode (will make API calls)")
            self.optimizer = PromptOptimizer(METRIC_CONFIG, CONFIG, api_key=api_key)
        
        print("✅ Demo system ready!")
    
    def _setup_mock_mode(self):
        """Setup mock evaluation and improvement methods."""
        
        # Mock evaluator
        def mock_eval(prompt, first_run=True, below_threshold_metrics=None):
            custom_results = self.optimizer.evaluator.custom_evaluator.evaluate_all(prompt)
            
            llm_metrics = {
                name: config for name, config in METRIC_CONFIG.items()
                if config['type'] == 'llm_judge'
            }
            if not first_run and below_threshold_metrics:
                llm_metrics = {
                    name: config for name, config in llm_metrics.items()
                    if name in below_threshold_metrics
                }
            
            llm_results = mock_llm_evaluation(prompt, list(llm_metrics.keys()))
            all_results = {**custom_results, **llm_results}
            return self.optimizer.evaluator._compile_results(prompt, all_results, first_run)
        
        self.optimizer.evaluator.evaluate = mock_eval
        
        # Mock improver
        def mock_improve(original, eval_result):
            return _mock_improve_prompt(original, eval_result)
        
        self.optimizer.improver.improve_prompt = mock_improve
    
    def run_optimization(self, prompt: str, max_iterations: int = None,
                        target_score: float = None, verbose: bool = True) -> Dict:
        """
        Run complete optimization on a prompt.
        
        Args:
            prompt: Initial prompt to optimize
            max_iterations: Max iterations (default: 5)
            target_score: Target score (default: 120)
            verbose: Print progress (default: True)
        
        Returns:
            Complete optimization results
        """
        if not verbose:
            # Suppress print statements
            import sys
            import io
            old_stdout = sys.stdout
            sys.stdout = io.StringIO()
        
        try:
            results = self.optimizer.optimize(
                prompt,
                max_iterations=max_iterations,
                target_score=target_score
            )
        finally:
            if not verbose:
                sys.stdout = old_stdout
        
        if verbose:
            self.print_summary(results)
        
        return results
    
    def print_summary(self, results: Dict):
        """Print optimization summary."""
        print("\n" + "🎯 "*40)
        print("OPTIMIZATION COMPLETE - SUMMARY")
        print("🎯 "*40)
        print(f"\n📊 Results:")
        print(f"   • Starting Score: {results['first_score']:.2f}/150 ({results['first_score']/150*100:.1f}%)")
        print(f"   • Best Score: {results['best_score']:.2f}/150 ({results['best_result']['percentage']:.1f}%)")
        print(f"   • Final Score: {results['final_score']:.2f}/150 ({results['final_score']/150*100:.1f}%)")
        print(f"   • Improvement: +{results['total_improvement']:.2f} points (+{results['improvement_percentage']:.1f}%)")
        print(f"   • Best Version: {results['best_result']['version']} (iteration {results['best_result']['iteration']})")
        print(f"   • Total Iterations: {results['total_iterations']}")
        print(f"\n{'✅ TARGET REACHED!' if results['target_reached'] else '⚠️  Target not reached'}")
    
    def get_best_prompt(self) -> str:
        """Get the best optimized prompt."""
        return self.optimizer.get_best_prompt()
    
    def visualize_all(self, results: Dict, save_path: str = None):
        """Create comprehensive visualization dashboard."""
        fig = self._create_dashboard(results)
        
        if save_path:
            fig.savefig(save_path, dpi=300, bbox_inches='tight')
            print(f"📊 Visualization saved to: {save_path}")
        
        plt.show()
        return fig
    
    def _create_dashboard(self, results: Dict) -> plt.Figure:
        """Create comprehensive dashboard."""
        fig = plt.figure(figsize=(20, 14))
        gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)
        
        # Title
        fig.suptitle(f'Prompt Optimization Dashboard - Best: {results["best_score"]:.1f}/150 '
                    f'({results["best_result"]["percentage"]:.1f}%)',
                    fontsize=18, fontweight='bold')
        
        history = results['history']
        
        # 1. Score Timeline (large, top)
        ax1 = fig.add_subplot(gs[0, :])
        iterations = [h['iteration'] for h in history]
        scores = [h['score'] for h in history]
        
        ax1.plot(iterations, scores, marker='o', linewidth=3, markersize=10, 
                color='steelblue', label='Score')
        ax1.axhline(y=CONFIG['target_score'], color='green', linestyle='--', 
                   linewidth=2, label=f'Target ({CONFIG["target_score"]})')
        
        # Highlight best
        best_iter = results['best_result']['iteration']
        best_score = results['best_result']['score']
        ax1.scatter([best_iter], [best_score], color='gold', s=500, marker='*', 
                   zorder=5, edgecolors='black', linewidth=2, label='Best')
        
        # Fill area
        ax1.fill_between(iterations, scores, alpha=0.3, color='steelblue')
        
        ax1.set_xlabel('Iteration', fontsize=12, fontweight='bold')
        ax1.set_ylabel('Total Score (out of 150)', fontsize=12, fontweight='bold')
        ax1.set_title('Score Progression Over Iterations', fontsize=14, fontweight='bold')
        ax1.grid(True, alpha=0.3)
        ax1.legend(fontsize=11)
        ax1.set_xticks(iterations)
        
        # 2. Metrics Below Threshold
        ax2 = fig.add_subplot(gs[1, 0])
        below_counts = [h['below_threshold_count'] for h in history]
        colors = ['#d32f2f' if c > 10 else '#ff9800' if c > 5 else '#4caf50' for c in below_counts]
        bars = ax2.bar(iterations, below_counts, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
        ax2.set_xlabel('Iteration', fontsize=11, fontweight='bold')
        ax2.set_ylabel('Count', fontsize=11, fontweight='bold')
        ax2.set_title('Metrics Below Threshold', fontsize=12, fontweight='bold')
        ax2.grid(True, alpha=0.3, axis='y')
        ax2.set_xticks(iterations)
        
        # Add value labels
        for bar, count in zip(bars, below_counts):
            height = bar.get_height()
            if height > 0:
                ax2.text(bar.get_x() + bar.get_width()/2., height,
                        f'{int(count)}', ha='center', va='bottom', fontweight='bold')
        
        # 3. Score Changes
        ax3 = fig.add_subplot(gs[1, 1])
        score_changes = [0]
        for i in range(1, len(history)):
            change = history[i]['score'] - history[i-1]['score']
            score_changes.append(change)
        
        colors_change = ['#4caf50' if c > 0 else '#d32f2f' if c < 0 else '#757575' 
                        for c in score_changes]
        bars = ax3.bar(iterations, score_changes, color=colors_change, alpha=0.8, 
                      edgecolor='black', linewidth=1.5)
        ax3.axhline(y=0, color='black', linestyle='-', linewidth=1)
        ax3.set_xlabel('Iteration', fontsize=11, fontweight='bold')
        ax3.set_ylabel('Score Change', fontsize=11, fontweight='bold')
        ax3.set_title('Score Change Per Iteration', fontsize=12, fontweight='bold')
        ax3.grid(True, alpha=0.3, axis='y')
        ax3.set_xticks(iterations)
        
        # Add value labels
        for bar, change in zip(bars, score_changes):
            if change != 0:
                height = bar.get_height()
                label_y = height + (0.3 if height > 0 else -0.8)
                ax3.text(bar.get_x() + bar.get_width()/2., label_y,
                        f'{change:+.1f}', ha='center', 
                        va='bottom' if height > 0 else 'top',
                        fontweight='bold', fontsize=9)
        
        # 4. Overall Score Gauge
        ax4 = fig.add_subplot(gs[1, 2])
        self._draw_gauge(ax4, results['best_score'], 150, 
                        title='Best Score', subtitle=results['best_result']['version'])
        
        # 5. Group Scores Heatmap
        ax5 = fig.add_subplot(gs[2, :2])
        best_eval = results['best_result']['evaluation']
        groups = list(best_eval['group_scores'].keys())
        group_percentages = [best_eval['group_scores'][g]['percentage'] for g in groups]
        
        colors_heat = []
        for pct in group_percentages:
            if pct >= 80:
                colors_heat.append('#1b5e20')
            elif pct >= 70:
                colors_heat.append('#4caf50')
            elif pct >= 50:
                colors_heat.append('#ff9800')
            else:
                colors_heat.append('#d32f2f')
        
        y_pos = range(len(groups))
        bars = ax5.barh(y_pos, group_percentages, color=colors_heat, alpha=0.8, 
                       edgecolor='black', linewidth=1.5)
        ax5.axvline(x=70, color='red', linestyle='--', linewidth=2, label='Threshold (70%)')
        ax5.set_yticks(y_pos)
        ax5.set_yticklabels([g.replace('_', ' ').title() for g in groups], fontsize=11)
        ax5.set_xlabel('Percentage', fontsize=11, fontweight='bold')
        ax5.set_title(f'Group Scores - Best Version ({results["best_result"]["version"]})', 
                     fontsize=12, fontweight='bold')
        ax5.legend(fontsize=10)
        ax5.grid(True, alpha=0.3, axis='x')
        ax5.set_xlim(0, 100)
        
        # Add percentage labels
        for bar, pct in zip(bars, group_percentages):
            width = bar.get_width()
            ax5.text(width + 2, bar.get_y() + bar.get_height()/2.,
                    f'{pct:.1f}%', ha='left', va='center', fontweight='bold', fontsize=10)
        
        # 6. Improvement Statistics
        ax6 = fig.add_subplot(gs[2, 2])
        ax6.axis('off')
        
        stats_text = f"""
        📈 IMPROVEMENT STATS
        
        Starting: {results['first_score']:.1f}/150
        ({results['first_score']/150*100:.1f}%)
        
        Best: {results['best_score']:.1f}/150
        ({results['best_result']['percentage']:.1f}%)
        
        Improvement: +{results['total_improvement']:.1f}
        (+{results['improvement_percentage']:.1f}%)
        
        Iterations: {results['total_iterations']}
        
        Best at: {results['best_result']['version']}
        
        Status: {'✅ Target Reached' if results['target_reached'] else '⚠️ In Progress'}
        """
        
        ax6.text(0.1, 0.5, stats_text, fontsize=11, verticalalignment='center',
                family='monospace', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
        
        return fig
    
    def _draw_gauge(self, ax, value, max_value, title="", subtitle=""):
        """Draw a gauge chart."""
        ax.set_xlim(0, 10)
        ax.set_ylim(0, 10)
        ax.axis('off')
        
        # Background circle
        circle_bg = plt.Circle((5, 5), 3, color='lightgray', fill=True, alpha=0.3)
        ax.add_patch(circle_bg)
        
        # Color based on percentage
        percentage = (value / max_value) * 100
        if percentage >= 80:
            color = '#4caf50'
        elif percentage >= 60:
            color = '#ff9800'
        else:
            color = '#d32f2f'
        
        # Value circle
        circle_val = plt.Circle((5, 5), 2.5, color=color, fill=True, alpha=0.8)
        ax.add_patch(circle_val)
        
        # Text
        ax.text(5, 5.5, f"{value:.1f}", ha='center', va='center', 
               fontsize=24, fontweight='bold', color='white')
        ax.text(5, 4.2, f"/ {max_value}", ha='center', va='center', 
               fontsize=14, color='white')
        
        if title:
            ax.text(5, 8.5, title, ha='center', va='center', 
                   fontsize=12, fontweight='bold')
        if subtitle:
            ax.text(5, 1.5, subtitle, ha='center', va='center', 
                   fontsize=10, style='italic')
    
    def export_results(self, results: Dict, output_dir: str = "./optimization_results"):
        """Export all results to files."""
        
        # Create output directory
        output_path = Path(output_dir)
        output_path.mkdir(exist_ok=True)
        
        print(f"\n📁 Exporting results to: {output_path.absolute()}")
        
        # 1. Save best prompt
        best_prompt_file = output_path / "best_prompt.txt"
        with open(best_prompt_file, 'w') as f:
            f.write(results['best_result']['prompt'])
        print(f"   ✅ Best prompt saved: {best_prompt_file.name}")
        
        # 2. Save full report
        report = self.optimizer.generate_optimization_report(results)
        report_file = output_path / "optimization_report.txt"
        with open(report_file, 'w') as f:
            f.write(report)
        print(f"   ✅ Full report saved: {report_file.name}")
        
        # 3. Save iteration history as CSV
        history_data = []
        for h in results['history']:
            history_data.append({
                'iteration': h['iteration'],
                'version': h['version'],
                'score': h['score'],
                'percentage': h['percentage'],
                'below_threshold_count': h['below_threshold_count']
            })
        
        df = pd.DataFrame(history_data)
        csv_file = output_path / "iteration_history.csv"
        df.to_csv(csv_file, index=False)
        print(f"   ✅ History saved: {csv_file.name}")
        
        # 4. Save all prompts
        prompts_dir = output_path / "prompts"
        prompts_dir.mkdir(exist_ok=True)
        
        for h in results['history']:
            prompt_file = prompts_dir / f"{h['version']}_prompt.txt"
            with open(prompt_file, 'w') as f:
                f.write(h['prompt'])
        print(f"   ✅ All prompts saved: {prompts_dir.name}/")
        
        # 5. Save visualization
        fig = self._create_dashboard(results)
        viz_file = output_path / "optimization_dashboard.png"
        fig.savefig(viz_file, dpi=300, bbox_inches='tight')
        plt.close(fig)
        print(f"   ✅ Dashboard saved: {viz_file.name}")
        
        # 6. Save JSON results
        import json
        
        # Create JSON-serializable version
        json_results = {
            'total_iterations': results['total_iterations'],
            'first_score': results['first_score'],
            'final_score': results['final_score'],
            'best_score': results['best_score'],
            'total_improvement': results['total_improvement'],
            'improvement_percentage': results['improvement_percentage'],
            'target_reached': results['target_reached'],
            'best_version': results['best_result']['version'],
            'best_iteration': results['best_result']['iteration'],
            'history': history_data
        }
        
        json_file = output_path / "results.json"
        with open(json_file, 'w') as f:
            json.dump(json_results, f, indent=2)
        print(f"   ✅ JSON results saved: {json_file.name}")
        
        print(f"\n✅ All results exported successfully!")
        print(f"📂 Location: {output_path.absolute()}")
        
        return output_path


print("✅ Demo Package Created!")

In [ ]:
def run_complete_demo():
    """
    Run a complete demonstration of the prompt optimization system.
    """
    
    print("="*100)
    print(" "*30 + "🚀 PROMPT OPTIMIZATION SYSTEM DEMO 🚀")
    print("="*100)
    print()
    print("This demo will:")
    print("  1. Initialize the optimization system")
    print("  2. Evaluate an initial prompt (scoring 0-150)")
    print("  3. Iteratively improve the prompt")
    print("  4. Track and return the best version")
    print("  5. Generate comprehensive reports and visualizations")
    print()
    print("="*100)
    input("Press ENTER to begin demo...")
    
    # Demo prompts
    demo_prompts = {
        "minimal": "Explain quantum computing.",
        
        "basic": """
Write an article about artificial intelligence.
Include recent developments and future trends.
Target audience: general readers.
""",
        
        "moderate": """
Create a technical guide about transformer models in NLP.

Requirements:
- Explain the architecture
- Compare BERT vs GPT
- Include practical use cases
- Keep it under 2000 words
"""
    }
    
    # Let user choose
    print("\n📝 Available test prompts:")
    for i, (name, prompt) in enumerate(demo_prompts.items(), 1):
        preview = prompt[:80].replace('\n', ' ')
        print(f"  {i}. {name}: {preview}...")
    
    choice = input("\nSelect prompt (1-3) or press ENTER for 'basic': ").strip()
    
    if choice == "1":
        prompt_name = "minimal"
    elif choice == "3":
        prompt_name = "moderate"
    else:
        prompt_name = "basic"
    
    test_prompt = demo_prompts[prompt_name]
    
    print(f"\n✅ Using '{prompt_name}' prompt")
    print(f"\nInitial Prompt:")
    print("─" * 100)
    print(test_prompt)
    print("─" * 100)
    
    input("\nPress ENTER to start optimization...")
    
    # Initialize demo system
    print("\n" + "="*100)
    demo = PromptOptimizationDemo(use_mock=True)
    
    # Run optimization
    print("\n" + "="*100)
    print("STARTING OPTIMIZATION")
    print("="*100)
    
    results = demo.run_optimization(
        test_prompt,
        max_iterations=5,
        target_score=120,
        verbose=True
    )
    
    # Show best prompt
    print("\n" + "="*100)
    print("BEST OPTIMIZED PROMPT")
    print("="*100)
    best_prompt = demo.get_best_prompt()
    print(best_prompt)
    print()
    
    # Visualize
    input("\nPress ENTER to see visualization...")
    demo.visualize_all(results)
    
    # Export option
    export = input("\n💾 Export results to files? (y/n): ").strip().lower()
    if export == 'y':
        output_dir = input("Enter output directory (default: ./optimization_results): ").strip()
        if not output_dir:
            output_dir = "./optimization_results"
        demo.export_results(results, output_dir)
    
    print("\n" + "="*100)
    print("✅ DEMO COMPLETE!")
    print("="*100)
    print(f"\n📊 Summary:")
    print(f"   • Iterations: {results['total_iterations']}")
    print(f"   • Improvement: {results['first_score']:.1f} → {results['best_score']:.1f} "
          f"(+{results['total_improvement']:.1f} points)")
    print(f"   • Best Version: {results['best_result']['version']}")
    print(f"   • Target Reached: {'Yes ✅' if results['target_reached'] else 'No ⚠️'}")
    
    return demo, results


# Optional: Quick start functions for different use cases
def quick_evaluate(prompt: str, use_mock: bool = True, api_key: str = None):
    """Quick evaluation of a single prompt."""
    demo = PromptOptimizationDemo(api_key=api_key, use_mock=use_mock)
    evaluator = demo.optimizer.evaluator
    result = evaluator.evaluate(prompt, first_run=True)
    report = evaluator.generate_detailed_report(result)
    print(report)
    return result


def quick_optimize(prompt: str, use_mock: bool = True, api_key: str = None):
    """Quick optimization with default settings."""
    demo = PromptOptimizationDemo(api_key=api_key, use_mock=use_mock)
    results = demo.run_optimization(prompt)
    demo.visualize_all(results)
    return demo, results


print("✅ Demo Script Ready!")
print("\n" + "="*100)
print("🎬 READY TO RUN DEMO")
print("="*100)
print("\nAvailable functions:")
print("  • run_complete_demo() - Interactive full demo")
print("  • quick_evaluate(prompt) - Just evaluate a prompt")
print("  • quick_optimize(prompt) - Optimize with defaults")
print("\nOr use the PromptOptimizationDemo class directly:")
print("  demo = PromptOptimizationDemo(use_mock=True)")
print("  results = demo.run_optimization(your_prompt)")
print("  demo.visualize_all(results)")
print("  demo.export_results(results)")
print("="*100)

In [ ]:
# Create a quick reference card
def print_quick_reference():
    """Print quick reference guide."""
    
    reference = """
    ╔═══════════════════════════════════════════════════════════════════════════════╗
    ║                     PROMPT OPTIMIZATION SYSTEM - QUICK REFERENCE              ║
    ╚═══════════════════════════════════════════════════════════════════════════════╝
    
    📊 SCORING SYSTEM
    ─────────────────────────────────────────────────────────────────────────────────
    Total Score: 0-150 points
    
    Group Distribution:
      • Structure & Clarity        25 pts (5 metrics × 5 pts)
      • Context & Information      25 pts (5 metrics × 5 pts)
      • Reasoning & Cognition      30 pts (6 metrics × 5 pts)  ← Highest priority
      • Safety & Alignment         30 pts (6 metrics × 5 pts)  ← Highest priority
      • Format & Style             15 pts (5 metrics × 3 pts)
      • Output Quality             20 pts (6 metrics × 3.33 pts)
      • Advanced Features           5 pts (3 metrics × 1.67 pts)
    
    Threshold: 70% of max points per metric
    Target: 120/150 (80%)
    
    
    🔧 BASIC USAGE
    ─────────────────────────────────────────────────────────────────────────────────
    
    # Initialize
    demo = PromptOptimizationDemo(use_mock=True)  # or use_mock=False with api_key
    
    # Optimize
    results = demo.run_optimization(
        prompt="Your prompt here",
        max_iterations=5,
        target_score=120
    )
    
    # Get best prompt
    best_prompt = demo.get_best_prompt()
    
    # Visualize
    demo.visualize_all(results)
    
    # Export
    demo.export_results(results, "./my_results")
    
    
    📈 OPTIMIZATION FLOW
    ─────────────────────────────────────────────────────────────────────────────────
    
    1. Initial Evaluation (all 37 metrics)
       ├─ Custom Functions (10 metrics) - pattern matching
       └─ LLM Judge (27 metrics) - Claude API
    
    2. Generate Improvement
       └─ Target ALL below-threshold metrics
    
    3. Re-Evaluate (only below-threshold metrics)
    
    4. Track Best (across all iterations)
    
    5. Repeat until:
       ├─ Target score reached (120/150)
       ├─ Max iterations (5)
       └─ Minimal improvement (<3 pts)
    
    
    🎯 KEY FEATURES
    ─────────────────────────────────────────────────────────────────────────────────
    
    ✅ Comprehensive Evaluation: 37 metrics across 7 categories
    ✅ Iterative Improvement: Automatic prompt enhancement
    ✅ Best Tracking: Always returns highest-scoring version
    ✅ Efficient Re-evaluation: Only re-checks improved metrics
    ✅ Mock Mode: Test without API calls
    ✅ Visualization: Interactive charts and dashboards
    ✅ Export: Reports, prompts, and data in multiple formats
    
    
    💡 TIPS
    ─────────────────────────────────────────────────────────────────────────────────
    
    • Start with mock mode to understand the system
    • Use real API for production optimization
    • Review improvement explanations to learn patterns
    • Best prompt may not be the final iteration (system tracks this!)
    • Export results to track progress over time
    
    
    📞 NEED HELP?
    ─────────────────────────────────────────────────────────────────────────────────
    
    run_complete_demo()      - Interactive guided demo
    quick_evaluate(prompt)   - Just evaluate, don't optimize
    quick_optimize(prompt)   - Quick optimization with defaults
    print_quick_reference()  - Show this guide again
    
    ═══════════════════════════════════════════════════════════════════════════════
    """
    
    print(reference)

# Print on load
print_quick_reference()

# FINAL SUMMARY

In [ ]:
def generate_final_summary_document():
    """Generate comprehensive final summary document."""
    
    summary = """
╔═══════════════════════════════════════════════════════════════════════════════╗
║                                                                               ║
║              PROMPT EVALUATION & OPTIMIZATION SYSTEM                          ║
║                         FINAL SUMMARY DOCUMENT                                ║
║                                                                               ║
╚═══════════════════════════════════════════════════════════════════════════════╝


═══════════════════════════════════════════════════════════════════════════════
📋 EXECUTIVE SUMMARY
═══════════════════════════════════════════════════════════════════════════════

SYSTEM PURPOSE:
A scientific, data-driven system for evaluating and optimizing prompts through
iterative refinement. The system measures prompts across 37 quality dimensions,
identifies weaknesses, generates improvements, and tracks the best version across
all iterations.

KEY CAPABILITIES:
✅ Comprehensive evaluation on 0-150 point scale
✅ 37 metrics across 7 quality dimensions
✅ Automated iterative improvement
✅ Best-prompt tracking (prevents regression)
✅ Dual evaluation approach (pattern matching + LLM judge)
✅ Visualization and reporting
✅ Mock mode for testing without API costs


═══════════════════════════════════════════════════════════════════════════════
🏗️ SYSTEM ARCHITECTURE
═══════════════════════════════════════════════════════════════════════════════

COMPONENTS:
┌─────────────────────────────────────────────────────────────────────────────┐
│ 1. METRIC CONFIGURATION                                                     │
│    • 37 metrics organized into 7 groups                                     │
│    • Group-based point allocation (150 total)                               │
│    • 70% threshold per metric                                               │
│    • Prioritization: Reasoning & Safety (30 pts each)                       │
└─────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────┐
│ 2. CUSTOM FUNCTION EVALUATOR                                                │
│    • 10 metrics via pattern matching                                        │
│    • Rule-based detection (regex, keyword analysis)                         │
│    • Instant evaluation, no API calls                                       │
│    • Metrics: Structure, Examples, Roles, Validation, etc.                  │
└─────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────┐
│ 3. LLM JUDGE EVALUATOR                                                      │
│    • 27 metrics via Claude API                                              │
│    • Subjective quality assessment                                          │
│    • Batched evaluation (5 metrics per call)                                │
│    • Metrics: Clarity, Reasoning, Safety, Alignment, etc.                   │
└─────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────┐
│ 4. FULL EVALUATOR PIPELINE                                                  │
│    • Combines both evaluator types                                          │
│    • Calculates total score (0-150)                                         │
│    • Identifies below-threshold metrics                                     │
│    • Group-level analysis                                                   │
│    • Generates detailed reports                                             │
└─────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────┐
│ 5. PROMPT IMPROVER                                                          │
│    • Targets ALL below-threshold metrics                                    │
│    • Uses Claude API for intelligent improvements                           │
│    • Preserves original intent                                              │
│    • Returns: improved prompt + change explanations                         │
└─────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────┐
│ 6. ITERATION MANAGER (PromptOptimizer)                                      │
│    • Orchestrates evaluation → improvement → re-evaluation                  │
│    • Tracks best prompt across ALL iterations                               │
│    • Efficient re-evaluation (only below-threshold metrics)                 │
│    • Multiple stopping criteria                                             │
│    • Comprehensive history tracking                                         │
└─────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────┐
│ 7. DEMO PACKAGE (PromptOptimizationDemo)                                    │
│    • User-friendly interface                                                │
│    • Mock mode for testing                                                  │
│    • Visualization dashboard                                                │
│    • Multi-format export                                                    │
│    • Quick-start functions                                                  │
└─────────────────────────────────────────────────────────────────────────────┘


═══════════════════════════════════════════════════════════════════════════════
📊 SCORING SYSTEM DETAILS
═══════════════════════════════════════════════════════════════════════════════

TOTAL: 150 POINTS

GROUP BREAKDOWN:
┌────────────────────────────┬────────┬──────────┬────────────┬──────────────┐
│ Group                      │ Points │ Metrics  │ Pts/Metric │ Priority     │
├────────────────────────────┼────────┼──────────┼────────────┼──────────────┤
│ Structure & Clarity        │   25   │    5     │    5.0     │ Medium       │
│ Context & Information      │   25   │    5     │    5.0     │ Medium       │
│ Reasoning & Cognition      │   30   │    6     │    5.0     │ ⭐ HIGHEST   │
│ Safety & Alignment         │   30   │    6     │    5.0     │ ⭐ HIGHEST   │
│ Format & Style             │   15   │    5     │    3.0     │ Low          │
│ Output Quality             │   20   │    6     │    3.33    │ Medium       │
│ Advanced Features          │    5   │    3     │    1.67    │ Low          │
└────────────────────────────┴────────┴──────────┴────────────┴──────────────┘

EVALUATION TYPES:
- Custom Functions: 10 metrics (instant, pattern-based)
- LLM Judge: 27 metrics (API-based, subjective assessment)

THRESHOLDS:
- Per-Metric Threshold: 70% of maximum points
- Target Score: 120/150 (80%)
- Minimum Improvement: 3 points to continue iterating


═══════════════════════════════════════════════════════════════════════════════
🔄 OPTIMIZATION WORKFLOW
═══════════════════════════════════════════════════════════════════════════════

ITERATION 1 (First Run):
├─ Evaluate ALL 37 metrics
│  ├─ Custom functions: 10 metrics
│  └─ LLM judge: 27 metrics (in batches of 5)
├─ Calculate total score
├─ Identify below-threshold metrics
└─ Generate improved prompt (targeting ALL below-threshold)

ITERATION 2-N (Subsequent Runs):
├─ Evaluate ONLY below-threshold metrics from previous iteration
│  └─ Efficiency: Skip metrics already above threshold
├─ Calculate total score
├─ Update best prompt if score improved
├─ Check stopping conditions:
│  ├─ Target score reached (120/150)
│  ├─ All metrics above threshold
│  ├─ Max iterations reached (5)
│  └─ Minimal improvement (<3 points)
└─ If continuing: Generate next improvement

FINAL OUTPUT:
└─ Return BEST prompt from ANY iteration (not just final)
   ├─ Prevents regression
   └─ Scientific approach: track highest quality achieved


═══════════════════════════════════════════════════════════════════════════════
🎯 KEY DESIGN DECISIONS & RATIONALE
═══════════════════════════════════════════════════════════════════════════════

1. GROUP-BASED SCORING (Not Equal Weight)
   Rationale: Prioritizes critical areas like reasoning and safety
   Impact: Guides improvement toward most important quality dimensions

2. DUAL EVALUATION APPROACH
   Rationale: Combines speed (custom) with depth (LLM judge)
   Impact: Faster, cheaper, yet comprehensive evaluation

3. BATCHED LLM EVALUATION
   Rationale: Reduces API calls while maintaining quality
   Impact: 27 metrics evaluated in ~6 API calls instead of 27

4. RE-EVALUATE ONLY BELOW-THRESHOLD
   Rationale: Efficiency - don't re-check what's already good
   Impact: Subsequent iterations much faster than first run

5. BEST PROMPT TRACKING
   Rationale: Iterative improvement can sometimes regress
   Impact: Always return the highest-quality version achieved

6. 70% THRESHOLD PER METRIC
   Rationale: Balance between achievability and quality
   Impact: Prompts must be "good" not just "present" on each dimension

7. TARGET ALL BELOW-THRESHOLD SIMULTANEOUSLY
   Rationale: Holistic improvement vs. incremental fixing
   Impact: Faster convergence to high-quality prompts


═══════════════════════════════════════════════════════════════════════════════
📈 EXPECTED PERFORMANCE
═══════════════════════════════════════════════════════════════════════════════

TYPICAL OPTIMIZATION TRAJECTORY:

Initial Prompt (Minimal):
└─ Score: 40-60/150 (27-40%)
   Many structural and reasoning elements missing

After 1 Iteration:
└─ Score: 70-90/150 (47-60%)
   Basic structure, examples, and clarity added

After 2-3 Iterations:
└─ Score: 95-115/150 (63-77%)
   Refinement of reasoning, safety, and completeness

After 4-5 Iterations:
└─ Score: 110-130/150 (73-87%)
   Polished, comprehensive prompts

API USAGE (Real Mode):
- First iteration: ~8-10 API calls
- Subsequent iterations: ~2-5 API calls (depending on metrics improved)
- Total for 5 iterations: ~20-30 API calls

TIME ESTIMATE:
- Mock mode: ~30 seconds for 5 iterations
- Real mode: ~2-4 minutes for 5 iterations (API latency)


═══════════════════════════════════════════════════════════════════════════════
🛠️ USAGE EXAMPLES
═══════════════════════════════════════════════════════════════════════════════

BASIC USAGE:
───────────────────────────────────────────────────────────────────────────────
from prompt_optimizer import PromptOptimizationDemo

# Initialize (mock mode - no API calls)
demo = PromptOptimizationDemo(use_mock=True)

# Optimize a prompt
results = demo.run_optimization(
    prompt="Write a blog post about AI",
    max_iterations=5,
    target_score=120
)

# Get best prompt
best = demo.get_best_prompt()
print(best)

# Visualize
demo.visualize_all(results)

# Export everything
demo.export_results(results, "./my_results")


PRODUCTION USAGE (With API):
───────────────────────────────────────────────────────────────────────────────
# Initialize with API key
demo = PromptOptimizationDemo(
    api_key="sk-ant-...",
    use_mock=False
)

# Optimize
results = demo.run_optimization(
    prompt=your_prompt,
    max_iterations=3,
    target_score=120
)


QUICK EVALUATION (No Optimization):
───────────────────────────────────────────────────────────────────────────────
result = quick_evaluate("Your prompt here", use_mock=True)
# Returns evaluation report


INTERACTIVE DEMO:
───────────────────────────────────────────────────────────────────────────────
run_complete_demo()
# Guided walkthrough with visualization


═══════════════════════════════════════════════════════════════════════════════
📦 OUTPUT FORMATS
═══════════════════════════════════════════════════════════════════════════════

EXPORTED FILES:
├── best_prompt.txt              (The highest-scoring prompt)
├── optimization_report.txt      (Complete iteration history & analysis)
├── iteration_history.csv        (Tabular data for external analysis)
├── results.json                 (Machine-readable results)
├── optimization_dashboard.png   (Visualization)
└── prompts/
    ├── v1_prompt.txt
    ├── v2_prompt.txt
    ├── v3_prompt.txt
    └── ...

VISUALIZATION DASHBOARD INCLUDES:
- Score progression timeline with best marker
- Below-threshold metrics count per iteration
- Score change per iteration (improvement/regression)
- Overall score gauge
- Group performance heatmap
- Detailed statistics panel


═══════════════════════════════════════════════════════════════════════════════
✅ SUCCESS CRITERIA (MET)
═══════════════════════════════════════════════════════════════════════════════

✓ Scientific Process
  → Measurable metrics, data-driven improvements, documented methodology

✓ 0-150 Scoring Scale
  → Group-based point allocation with 37 metrics

✓ Iterative Optimization
  → Automatic improvement loop with multiple stopping criteria

✓ Best Prompt Tracking
  → System always returns highest-scoring version

✓ Efficiency
  → First run: all metrics; Subsequent: only below-threshold

✓ Demo-Ready
  → Complete demo package with visualization and export

✓ Extensible
  → Easy to add new metrics or modify scoring

✓ Mock Mode
  → Test without API costs


═══════════════════════════════════════════════════════════════════════════════
🔮 FUTURE ENHANCEMENTS (Optional)
═══════════════════════════════════════════════════════════════════════════════

Possible Extensions:
- Multi-prompt comparison mode
- Custom metric weights by use case
- A/B testing framework
- Integration with prompt libraries
- Real-time evaluation API endpoint
- Batch processing for multiple prompts
- Historical trend analysis
- Domain-specific metric sets


═══════════════════════════════════════════════════════════════════════════════
📞 SUPPORT & REFERENCE
═══════════════════════════════════════════════════════════════════════════════

QUICK COMMANDS:
- run_complete_demo()           - Full interactive demo
- quick_evaluate(prompt)        - Evaluate only
- quick_optimize(prompt)        - Optimize with defaults
- print_quick_reference()       - Show quick reference

DOCUMENTATION:
- All code is documented with docstrings
- Inline comments explain complex logic
- Example usage throughout

KEY CLASSES:
- PromptOptimizationDemo        - Main interface
- PromptOptimizer               - Core orchestrator
- PromptEvaluator               - Unified evaluation
- PromptImprover                - Improvement generation
- CustomMetricEvaluator         - Pattern-based metrics
- LLMJudgeEvaluator            - LLM-based metrics


═══════════════════════════════════════════════════════════════════════════════
🎉 CONCLUSION
═══════════════════════════════════════════════════════════════════════════════

This system provides a complete, scientific approach to prompt optimization:

✅ Comprehensive evaluation across 37 quality dimensions
✅ Intelligent iterative improvement
✅ Best-version tracking (no regression)
✅ Production-ready with mock testing capability
✅ Extensive visualization and reporting
✅ Easy to use, easy to extend

The system successfully transforms vague prompts into high-quality, well-
structured prompts through measurable, data-driven iteration.

READY FOR DEMONSTRATION AND DEPLOYMENT! 🚀


═══════════════════════════════════════════════════════════════════════════════
Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
═══════════════════════════════════════════════════════════════════════════════
"""
    
    return summary

# Generate and display summary
summary_doc = generate_final_summary_document()
print(summary_doc)

# Save summary to file
with open("SYSTEM_SUMMARY.txt", "w") as f:
    f.write(summary_doc)

print("\n✅ Summary document saved to: SYSTEM_SUMMARY.txt")

In [ ]:
print("\n\n" + "="*100)
print("="*100)
print(" "*35 + "🧪 RUNNING TEST OPTIMIZATION 🧪")
print("="*100)
print("="*100)

# Initialize demo system
print("\n📦 Initializing system...")
demo = PromptOptimizationDemo(use_mock=True)

# Test prompt - intentionally basic to show improvement
test_prompt = """
Write a technical article about machine learning for beginners.
Include some examples and keep it interesting.
"""

print("\n" + "="*100)
print("INITIAL PROMPT")
print("="*100)
print(test_prompt)
print()

# Run optimization
print("="*100)
print("STARTING OPTIMIZATION (This will take ~30 seconds in mock mode)")
print("="*100)

import time
start_time = time.time()

results = demo.run_optimization(
    prompt=test_prompt,
    max_iterations=5,
    target_score=120,
    verbose=True
)

elapsed = time.time() - start_time

# Show results
print("\n" + "="*100)
print("🏆 BEST OPTIMIZED PROMPT 🏆")
print("="*100)
best_prompt = demo.get_best_prompt()
print(best_prompt)
print()

# Print iteration trajectory
print("="*100)
print("📈 OPTIMIZATION TRAJECTORY")
print("="*100)
trajectory = demo.optimizer.get_improvement_trajectory()
print(f"\n{'Iteration':<12} {'Version':<10} {'Score':<15} {'Percentage':<12} {'Below Threshold':<20}")
print("-" * 100)
for t in trajectory:
    print(f"{t['iteration']:<12} {t['version']:<10} {t['score']:<15.2f} {t['percentage']:<12.1f}% {t['below_threshold']:<20}")

# Show key improvements
print("\n" + "="*100)
print("🔍 KEY IMPROVEMENTS MADE")
print("="*100)

for i, iteration in enumerate(results['history']):
    if 'improvement' in iteration:
        print(f"\n{iteration['version']} → v{i+2}:")
        print("-" * 100)
        changes = iteration['improvement']['change_explanations']
        # Show first 500 chars of changes
        print(changes[:500] + "..." if len(changes) > 500 else changes)

# Statistics
print("\n" + "="*100)
print("📊 FINAL STATISTICS")
print("="*100)
print(f"""
Total Time: {elapsed:.2f} seconds
Total Iterations: {results['total_iterations']}

SCORES:
  Starting: {results['first_score']:.2f}/150 ({results['first_score']/150*100:.1f}%)
  Best:     {results['best_score']:.2f}/150 ({results['best_result']['percentage']:.1f}%)
  Final:    {results['final_score']:.2f}/150 ({results['final_score']/150*100:.1f}%)

IMPROVEMENT:
  Total Gain: +{results['total_improvement']:.2f} points
  Percentage: +{results['improvement_percentage']:.1f}%
  
BEST VERSION:
  Version: {results['best_result']['version']}
  Iteration: {results['best_result']['iteration']}
  
TARGET:
  Goal: 120/150 (80%)
  Achieved: {'✅ YES' if results['target_reached'] else '❌ NO'}

GROUP SCORES (Best Version):
""")

for group, data in results['best_result']['evaluation']['group_scores'].items():
    status = "✅" if data['percentage'] >= 70 else "⚠️"
    print(f"  {status} {group.replace('_', ' ').title():<30} {data['score']:>6.2f}/{data['max']:<6.0f} ({data['percentage']:>5.1f}%)")

# Visualize
print("\n" + "="*100)
print("📊 GENERATING VISUALIZATION")
print("="*100)
demo.visualize_all(results)

# Offer to export
print("\n" + "="*100)
print("💾 EXPORT OPTIONS")
print("="*100)
print("Results can be exported with: demo.export_results(results, './my_results')")
print("This will save:")
print("  • Best prompt (TXT)")
print("  • Full report (TXT)")
print("  • Iteration history (CSV)")
print("  • All prompts (TXT)")
print("  • Visualization (PNG)")
print("  • Results data (JSON)")

# Final summary
print("\n" + "="*100)
print("="*100)
print(" "*40 + "✅ TEST COMPLETE! ✅")
print("="*100)
print("="*100)

print(f"""
🎉 SYSTEM DEMONSTRATION SUCCESSFUL!

The prompt went from {results['first_score']:.1f} points to {results['best_score']:.1f} points 
through {results['total_iterations']} iterations of scientific optimization.

Key Achievements:
✓ Comprehensive evaluation (37 metrics)
✓ Iterative improvement
✓ Best version tracking
✓ Detailed reporting
✓ Visual progress tracking

The system is ready for production use! 🚀
""")

# Store results for further exploration
test_results = results
test_demo = demo

print("\n📌 Results stored in variables:")
print("   • test_results - Full optimization results")
print("   • test_demo - Demo instance (use for export)")
print("\nExample: test_demo.export_results(test_results, './my_optimization')")